In [ ]:
from datasets import load_dataset
from evaluate import load
from pathlib import Path
import os
import yaml
import re
import math

In [ ]:
path_to_predictions = r''  # path to .json file
path_to_save_metrics = r''

In [ ]:
dataset = load_dataset('json', data_files=path_to_predictions)['train']

In [ ]:
pattern = r'[a-z]+[0-9]*|[A-Z](?:[a-z]+[0-9]*|[A-Z]*[0-9]*(?=[A-Z]|$))'

def split(example):
    example['words_labels'] = ' '.join(re.findall(pattern, example['labels'])).lower()
    example['words_predictions'] = ' '.join(re.findall(pattern, example['prediction'])).lower()
    return example

In [ ]:
predictions_dataset = dataset.map(split)

In [ ]:
# for each example we extract the body from code (string between { and })
def extract_body(example):
    code = example['input_code']
    body = code[code.find('{') + 1:code.rfind('}')]
    example['body'] = body
    return example

In [ ]:
predictions_dataset = predictions_dataset.map(extract_body)

In [ ]:
# count the number of lines for each body
def count_lines(example):
    example['nb_lines'] = len(example['body'].split('\n')) - 2
    return example

In [ ]:
predictions_dataset = predictions_dataset.map(count_lines)

In [ ]:
ranges = [[0, 5, 0], [5, 10, 0], [10, 20, 0], [20, 50, 0], [50, 100, 0], [100, 200, 0], [200, 500, 0], [500, math.inf, 0]]

In [ ]:
# count the number of examples for each range
def count_examples(example):
    for r in ranges:
        if r[0] <= example['nb_lines'] < r[1]:
            r[2] += 1
            break

In [ ]:
predictions_dataset.map(count_examples)

In [ ]:
metrics = {}

In [ ]:
# for each range calculate the metrics
rouge = load('rouge')
exact_match_metric = load('exact_match')
# for each range we have a dict with the metrics
metrics['rouge'] =  {
    'rouge1': {},
    'rouge2': {},
    'rougeL': {},
    'rougeLsum': {}
}
metrics['exact_match'] = {}

for range in ranges:
    filtered_dataset = predictions_dataset.filter(lambda example: range[0] <= example['nb_lines'] < range[1])

    if len(filtered_dataset) == 0:
        for k, v in metrics['rouge'].items():
            v[f'{range[0]}-{range[1]}'] = 0
        continue

    rouge_results = rouge.compute(
        predictions=filtered_dataset['words_predictions'],
        references=filtered_dataset['words_labels'],
        use_stemmer=True)

    for k, v in rouge_results.items():
        metrics['rouge'][k][f'{range[0]}-{range[1]}'] = float(v)

    exact_match_results = exact_match_metric.compute(
        predictions=filtered_dataset['words_predictions'],
        references=filtered_dataset['words_labels'])

    metrics['exact_match'][f'{range[0]}-{range[1]}'] = float(exact_match_results['exact_match'])

In [ ]:
Path(path_to_save_metrics).mkdir(parents=True, exist_ok=True)

In [ ]:
with open(os.path.join(path_to_save_metrics, 'metrics.yml'), 'w') as f:
    yaml.dump(metrics, f)